In [1]:
import findspark

findspark.init()
findspark.find()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("spark://spark-master:7077").appName("test").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
[2023-09-26 05:41:52] {NativeCodeLoader} WARN: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
# spark.stop()

In [2]:
from os import getenv
import dotenv

dotenv.find_dotenv()
dotenv.load_dotenv()

False

In [3]:
spark = (
    SparkSession.builder.master("spark://spark-master:7077")
        .appName('test-app')
        .config(
            map={
                "spark.hadoop.fs.s3a.access.key": getenv("S3_ACCESS_KEY_ID"),
                "spark.hadoop.fs.s3a.secret.key": getenv("S3_SECRET_ACCESS_KEY"),
                "spark.hadoop.fs.s3a.endpoint": getenv("S3_ENDPOINT_URL"),
                "spark.hadoop.fs.s3a.impl": "org.apache.hadoop.fs.s3a.S3AFileSystem",
            }
        )
        .getOrCreate()
    )

[2023-09-26 05:41:56] {SparkSession} WARN: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
BOOTSTRAP_SERVER = (
    f"{getenv('YC_KAFKA_BROKER_HOST')}:{getenv('YC_KAFKA_BROKER_PORT')}"
)
USERNAME = getenv("YC_KAFKA_USERNAME")
PASSWORD = getenv("YC_KAFKA_PASSWORD")
CERTIFICATE_PATH = f'{getenv("APP_PATH")}/CA.pem'

if not all([BOOTSTRAP_SERVER, USERNAME, PASSWORD]):
    raise ValueError(
        "Required environment varialbes not set. See .env.template for more details"
    )
log.info(
    f"Reading stream with given config:\n {pformat(object=config, underscore_numbers=True)}"
)

options = {
    "kafka.bootstrap.servers": BOOTSTRAP_SERVER,
    "startingOffsets": "earliest",
    "subscribe": config["topic"],
    "kafka.security.protocol": "SASL_SSL",
    "kafka.sasl.mechanism": "SCRAM-SHA-512",
    "kafka.sasl.jaas.config": f'org.apache.kafka.common.security.scram.ScramLoginModule required username="{USERNAME}" password="{PASSWORD}";',
    "kafka.ssl.truststore.type": "PEM",
    "kafka.ssl.truststore.location": CERTIFICATE_PATH,
    "maxOffsetsPerTrigger": config["trigger"]["offsets-per-trigger"],
}

log.info(
    f"Subscribe to {config['topic']} kafka topic. Will consume {config['trigger']['offsets-per-trigger']} offsets per one trigger"
)

frame = (
    spark.readStream.format("kafka")
    .options(**options)
    .load()
    .select(
        F.col("value").cast(T.StringType()),
    )
    .withColumn(
        "value",
        F.from_json(
            col=F.col("value"),
            schema=T.StructType(
                [
                    T.StructField("object_id", T.StringType(), True),
                    T.StructField("object_type", T.StringType(), True),
                    T.StructField("sent_dttm", T.StringType(), True),
                    T.StructField("payload", T.StringType(), True),
                ]
            ),
        ),
    )
    .withColumns(
        dict(
            object_id=F.col("value.object_id"),
            object_type=F.col("value.object_type"),
            sent_dttm=F.col("value.sent_dttm"),
            payload=F.col("value.payload"),
        )
    )
    .withColumns(
        dict(
            sent_dttm=F.to_timestamp(
                F.regexp_replace(F.col("sent_dttm"), "T", " "),
                r"yyyy-MM-dd HH:mm:ss",
            ),
            object_type=F.lower(F.col("object_type")),
            trigger_dttm=F.lit(datetime.now()),
        )
    )
)